In [1]:
import pandas as pd
import numpy as np

## DNN

In [3]:
data_dnn = pd.read_json('results_dnn.json').T

data_dnn

,n_epochs,batch_size,learning_rate,n_layers,activation_function,loss_function,optimizer,results
0,50,32,0.001,3,relu,Cross Entropy,ADAM,"[{'loss_values': [232.88340389728546, 219.4525..."
1,50,32,0.001,3,relu,Cross Entropy,SGD,"[{'loss_values': [239.34742295742035, 239.1654..."
2,50,32,0.001,3,relu,Cross Entropy,RMSprop,"[{'loss_values': [1311.3456435203552, 228.9097..."
3,50,32,0.001,3,relu,Multi Margin,ADAM,"[{'loss_values': [90.5774205327034, 81.7014083..."
4,50,32,0.001,3,relu,Multi Margin,SGD,"[{'loss_values': [105.32916295528412, 105.0802..."
...,...,...,...,...,...,...,...,...
139,50,64,0.1,15,sigmoid,Cross Entropy,SGD,"[{'loss_values': [164.53463542461395, 124.4378..."
140,50,64,0.1,15,sigmoid,Cross Entropy,RMSprop,"[{'loss_values': [12514.1414488554, 5595.03428..."
141,50,64,0.1,15,sigmoid,Multi Margin,ADAM,"[{'loss_values': [504.3324379324913, 226.14446..."
142,50,64,0.1,15,sigmoid,Multi Margin,SGD,"[{'loss_values': [67.28866493701935, 59.539265..."


In [ ]:
metrics_dnn = []

for i, row in data_dnn.iterrows():

    accuracy_fold_train = []
    accuracy_fold_test = []

    recall_fold_train = []
    recall_fold_test = []

    precision_fold_train = []
    precision_fold_test = []

    for iteration in row["results"]:
        
        loss = iteration["loss_values"]
        conf_mat_train = np.array(iteration["confusion_matrix_train"])
        conf_mat_test = np.array(iteration["confusion_matrix_val"])
        
        # Compute accuracy (7 classes)
        total_train = conf_mat_train.sum()
        total_test = conf_mat_test.sum()

        accuracy_train = conf_mat_train.diagonal().sum() / total_train if total_train > 0 else 0
        accuracy_test = conf_mat_test.diagonal().sum() / total_test if total_test > 0 else 0

        # Compute recall and precision (per class)
        recall_train = np.array([
            conf_mat_train[i, i] / conf_mat_train[i, :].sum() if conf_mat_train[i, :].sum() > 0 else 0
            for i in range(7)
        ])
        recall_test = np.array([
            conf_mat_test[i, i] / conf_mat_test[i, :].sum() if conf_mat_test[i, :].sum() > 0 else 0
            for i in range(7)
        ])

        precision_train = np.array([
            conf_mat_train[i, i] / conf_mat_train[:, i].sum() if conf_mat_train[:, i].sum() > 0 else 0
            for i in range(7)
        ])
        precision_test = np.array([
            conf_mat_test[i, i] / conf_mat_test[:, i].sum() if conf_mat_test[:, i].sum() > 0 else 0
            for i in range(7)
        ])

        accuracy_fold_train.append(accuracy_train)
        accuracy_fold_test.append(accuracy_test)

        recall_fold_train.append(recall_train)
        recall_fold_test.append(recall_test)

        precision_fold_train.append(precision_train)
        precision_fold_test.append(precision_test)

    # Compute mean across folds
    accuracy_train = np.mean(accuracy_fold_train)
    accuracy_test = np.mean(accuracy_fold_test)

    recall_train = np.mean(recall_fold_train, axis=0)
    recall_test = np.mean(recall_fold_test, axis=0)

    precision_train = np.mean(precision_fold_train, axis=0)
    precision_test = np.mean(precision_fold_test, axis=0)

    average_recall_train = np.mean(recall_train)
    average_recall_test = np.mean(recall_test)

    average_precision_train = np.mean(precision_train)
    average_precision_test = np.mean(precision_test)

    f1_train_average = 2 * (average_precision_train * average_recall_train) / (average_precision_train + average_recall_train)
    f1_test_average = 2 * (average_precision_test * average_recall_test) / (average_precision_test + average_recall_test) 

    metrics_dnn.append({
        "iteration": i,
        "accuracy_train": accuracy_train,
        "accuracy_test": accuracy_test,
        "recall_train": recall_train.tolist(),  # Convert to list for DataFrame compatibility
        "recall_test": recall_test.tolist(),
        "averaged_recall_train": average_recall_train,
        "averaged_recall_test": average_recall_test,
        "precision_train": precision_train.tolist(),
        "precision_test": precision_test.tolist(),
        "averaged_precision_train": average_precision_train,
        "averaged_precision_test": average_precision_test,
        "f1_train_average": f1_train_average,
        "f1_test_average": f1_test_average
    })

metrics_dnn = pd.DataFrame(metrics_dnn)
metrics_dnn


,iteration,accuracy_train,accuracy_test,recall_train,recall_test,averaged_recall_train,averaged_recall_test,precision_train,precision_test,averaged_precision_train,averaged_precision_test,f1_train_average,f1_test_average
0,0,0.492194,0.452449,"[0.45666576580645024, 0.2781380793631726, 0.41...","[0.39492310575368617, 0.26124302397151344, 0.3...",0.492041,0.453465,"[0.3923431594126644, 0.4582671777499705, 0.382...","[0.35207534729986034, 0.3796717171717171, 0.35...",0.508026,0.459296,0.499906,0.456362
1,1,0.289694,0.276531,"[0.10461486097624861, 0.535960678336236, 0.091...","[0.0926535974196895, 0.540545522099994, 0.0831...",0.288489,0.281088,"[0.2850857940170572, 0.2471789073476965, 0.233...","[0.4244749380598437, 0.24591828080953, 0.17545...",0.330505,0.328967,0.308071,0.303149
2,2,0.469490,0.425714,"[0.22926222423077203, 0.2317780149045233, 0.46...","[0.17854807761175776, 0.18497423247048367, 0.4...",0.469501,0.425392,"[0.4713139798855225, 0.4143381572073497, 0.313...","[0.35781766024912004, 0.3630360986918364, 0.29...",0.492767,0.439270,0.480853,0.432220
3,3,0.500663,0.470204,"[0.42116714317564297, 0.3971881422840021, 0.37...","[0.3596384879068333, 0.3887226116726817, 0.346...",0.501354,0.467065,"[0.40255447100242014, 0.397145191949951, 0.459...","[0.3486435172522411, 0.38564581583879814, 0.41...",0.520917,0.484875,0.510949,0.475803
4,4,0.273878,0.260612,"[0.12795219917516099, 0.6912996932283679, 0.17...","[0.12190061903965199, 0.6954819776464157, 0.16...",0.271799,0.268520,"[0.22030669708290626, 0.2214708161247449, 0.13...","[0.10364346128822381, 0.22680118760984924, 0.1...",0.297552,0.239952,0.284093,0.253433
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,139,0.143418,0.140612,"[0.2, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0]","[0.2, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0]",0.142857,0.142857,"[0.028775510204081634, 0.0, 0.0, 0.0, 0.0, 0.1...","[0.027755102040816326, 0.0, 0.0, 0.0, 0.0, 0.1...",0.020488,0.020087,0.035837,0.035222
140,140,0.143265,0.141224,"[0.2, 0.2, 0.0, 0.2, 0.0, 0.4, 0.0]","[0.2, 0.2, 0.0, 0.2, 0.0, 0.4, 0.0]",0.142857,0.142857,"[0.028775510204081634, 0.02811224489795918, 0....","[0.027755102040816326, 0.03040816326530612, 0....",0.020466,0.020175,0.035804,0.035357
141,141,0.141837,0.146939,"[0.0, 0.2, 0.2, 0.0, 0.2, 0.2, 0.2]","[0.0, 0.2, 0.2, 0.0, 0.2, 0.2, 0.2]",0.142857,0.142857,"[0.0, 0.02857142857142857, 0.02806122448979591...","[0.0, 0.02857142857142857, 0.03061224489795918...",0.020262,0.020991,0.035491,0.036604
142,142,0.146122,0.129796,"[0.2, 0.2, 0.2, 0.0, 0.2, 0.2, 0.0]","[0.2, 0.2, 0.2, 0.0, 0.2, 0.2, 0.0]",0.142857,0.142857,"[0.028775510204081634, 0.028520408163265305, 0...","[0.027755102040816326, 0.028775510204081634, 0...",0.020875,0.018542,0.036427,0.032824


In [30]:
# get the max value for f1-score

max_f1_test = metrics_dnn["f1_test_average"].max()
max_f1_train = metrics_dnn["f1_train_average"].max()
max_f1_test, max_f1_train

(np.float64(0.49206776874351155), np.float64(0.5411958702807196))

## CNN

In [2]:
data_cnn = pd.read_json('results_cnn_1.json').T

data_cnn

,n_epochs,activation_function,pooling,n_conv_layers,conv_out_channels,conv_kernel_size,conv_padding,n_layers,batch_size,learning_rate,loss_function,optimizer,results
0,50,relu,MaxPool,1,16,3,1,3,32,0.001,Cross Entropy,ADAM,"[{'loss_values': [231.8701878786087, 197.12700..."
1,50,relu,MaxPool,1,16,3,1,3,32,0.001,Cross Entropy,SGD,"[{'loss_values': [239.3160616159439, 239.26000..."
2,50,relu,MaxPool,1,16,3,1,3,32,0.001,Cross Entropy,RMSprop,"[{'loss_values': [5110.887897610664, 220.26268..."
3,50,relu,MaxPool,1,16,3,1,3,32,0.001,Multi Margin,ADAM,"[{'loss_values': [86.5283233821392, 55.5496368..."
4,50,relu,MaxPool,1,16,3,1,3,32,0.001,Multi Margin,SGD,"[{'loss_values': [105.38043284416199, 105.3138..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,50,relu,MaxPool,2,16,3,1,15,32,0.001,Multi Margin,ADAM,"[{'loss_values': [105.79210501909256, 105.7310..."
112,50,relu,MaxPool,2,16,3,1,15,32,0.001,Multi Margin,SGD,"[{'loss_values': [105.42678338289261, 105.4261..."
113,50,relu,MaxPool,2,16,3,1,15,32,0.001,Multi Margin,RMSprop,"[{'loss_values': [133.8521836400032, 105.55857..."
114,50,relu,MaxPool,2,16,3,1,15,32,0.01,Cross Entropy,ADAM,"[{'loss_values': [652.23493039608, 239.5660724..."


In [5]:
metrics_cnn = []

for i, row in data_cnn.iterrows():

    accuracy_fold_train = []
    accuracy_fold_test = []

    recall_fold_train = []
    recall_fold_test = []

    precision_fold_train = []
    precision_fold_test = []

    for iteration in row["results"]:
        
        loss = iteration["loss_values"]
        conf_mat_train = np.array(iteration["confusion_matrix_train"])
        conf_mat_test = np.array(iteration["confusion_matrix_val"])
        
        # Compute accuracy (7 classes)
        total_train = conf_mat_train.sum()
        total_test = conf_mat_test.sum()

        accuracy_train = conf_mat_train.diagonal().sum() / total_train if total_train > 0 else 0
        accuracy_test = conf_mat_test.diagonal().sum() / total_test if total_test > 0 else 0

        # Compute recall and precision (per class)
        recall_train = np.array([
            conf_mat_train[i, i] / conf_mat_train[i, :].sum() if conf_mat_train[i, :].sum() > 0 else 0
            for i in range(7)
        ])
        recall_test = np.array([
            conf_mat_test[i, i] / conf_mat_test[i, :].sum() if conf_mat_test[i, :].sum() > 0 else 0
            for i in range(7)
        ])

        precision_train = np.array([
            conf_mat_train[i, i] / conf_mat_train[:, i].sum() if conf_mat_train[:, i].sum() > 0 else 0
            for i in range(7)
        ])
        precision_test = np.array([
            conf_mat_test[i, i] / conf_mat_test[:, i].sum() if conf_mat_test[:, i].sum() > 0 else 0
            for i in range(7)
        ])

        accuracy_fold_train.append(accuracy_train)
        accuracy_fold_test.append(accuracy_test)

        recall_fold_train.append(recall_train)
        recall_fold_test.append(recall_test)

        precision_fold_train.append(precision_train)
        precision_fold_test.append(precision_test)

    # Compute mean across folds
    accuracy_train = np.mean(accuracy_fold_train)
    accuracy_test = np.mean(accuracy_fold_test)

    recall_train = np.mean(recall_fold_train, axis=0)
    recall_test = np.mean(recall_fold_test, axis=0)

    precision_train = np.mean(precision_fold_train, axis=0)
    precision_test = np.mean(precision_fold_test, axis=0)

    average_recall_train = np.mean(recall_train)
    average_recall_test = np.mean(recall_test)

    average_precision_train = np.mean(precision_train)
    average_precision_test = np.mean(precision_test)

    f1_train_average = 2 * (average_precision_train * average_recall_train) / (average_precision_train + average_recall_train)
    f1_test_average = 2 * (average_precision_test * average_recall_test) / (average_precision_test + average_recall_test) 

    metrics_cnn.append({
        "iteration": i,
        "accuracy_train": accuracy_train,
        "accuracy_test": accuracy_test,
        "recall_train": recall_train.tolist(),  # Convert to list for DataFrame compatibility
        "recall_test": recall_test.tolist(),
        "averaged_recall_train": average_recall_train,
        "averaged_recall_test": average_recall_test,
        "precision_train": precision_train.tolist(),
        "precision_test": precision_test.tolist(),
        "averaged_precision_train": average_precision_train,
        "averaged_precision_test": average_precision_test,
        "f1_train_average": f1_train_average,
        "f1_test_average": f1_test_average
    })

metrics_cnn = pd.DataFrame(metrics_cnn)
metrics_cnn


,iteration,accuracy_train,accuracy_test,recall_train,recall_test,averaged_recall_train,averaged_recall_test,precision_train,precision_test,averaged_precision_train,averaged_precision_test,f1_train_average,f1_test_average
0,0,0.878571,0.587959,"[0.8783461695378652, 0.7727700035245286, 0.804...","[0.5332652149018743, 0.4189186747709572, 0.436...",0.878731,0.587685,"[0.8394743897181787, 0.8892373590590665, 0.857...","[0.5007983883078934, 0.5383121861435812, 0.491...",0.882657,0.588455,0.880689,0.588070
1,1,0.282704,0.272449,"[0.18116847287235294, 0.6546719473864446, 0.00...","[0.17621491506095824, 0.6553112634440799, 0.00...",0.280925,0.279807,"[0.23425496000797144, 0.23665133438647676, 0.1...","[0.18028448206330153, 0.23623675962678226, 0.0...",0.333323,0.311014,0.304889,0.294586
2,2,0.907041,0.608980,"[0.8266388045034054, 0.8962256848774655, 0.902...","[0.47887338277403957, 0.5030780299028847, 0.54...",0.907138,0.608596,"[0.9468313178098764, 0.8848190094839643, 0.835...","[0.6046804023160028, 0.547976948165036, 0.4816...",0.914522,0.616030,0.910815,0.612291
3,3,0.900000,0.589388,"[0.889013033613435, 0.8476563638588956, 0.8321...","[0.5350818736054895, 0.4777231375858392, 0.430...",0.900022,0.590088,"[0.8566739641952855, 0.8641449562608097, 0.916...","[0.5015345071595071, 0.5263866979086214, 0.514...",0.903775,0.589573,0.901895,0.589830
4,4,0.167449,0.152857,"[0.12558665033680344, 0.5996485061511423, 0.0,...","[0.1215151515151515, 0.6, 0.0, 0.0, 0.00859595...",0.164875,0.162998,"[0.05430857310628303, 0.09091184807202482, 0.0...","[0.046469622331691295, 0.08352534986527714, 0....",0.141456,0.105480,0.152270,0.128078
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,111,0.145969,0.130408,"[0.2, 0.2, 0.0, 0.2, 0.2, 0.0, 0.2]","[0.2, 0.2, 0.0, 0.2, 0.2, 0.0, 0.2]",0.142857,0.142857,"[0.029336734693877552, 0.02923469387755102, 0....","[0.025510204081632654, 0.025918367346938774, 0...",0.020853,0.018630,0.036393,0.032961
112,112,0.145510,0.132245,"[0.2, 0.4, 0.4, 0.0, 0.0, 0.0, 0.0]","[0.2, 0.4, 0.4, 0.0, 0.0, 0.0, 0.0]",0.142857,0.142857,"[0.029336734693877552, 0.05790816326530612, 0....","[0.025510204081632654, 0.05408163265306123, 0....",0.020787,0.018892,0.036293,0.033371
113,113,0.145969,0.130408,"[0.2, 0.2, 0.0, 0.2, 0.2, 0.0, 0.2]","[0.2, 0.2, 0.0, 0.2, 0.2, 0.0, 0.2]",0.142857,0.142857,"[0.029336734693877552, 0.02923469387755102, 0....","[0.025510204081632654, 0.025918367346938774, 0...",0.020853,0.018630,0.036393,0.032961
114,114,0.145459,0.132449,"[0.0, 0.2, 0.0, 0.4, 0.0, 0.0, 0.4]","[0.0, 0.2, 0.0, 0.4, 0.0, 0.0, 0.4]",0.142857,0.142857,"[0.0, 0.02923469387755102, 0.0, 0.058163265306...","[0.0, 0.025918367346938774, 0.0, 0.05306122448...",0.020780,0.018921,0.036282,0.033417


In [8]:
max_f1_train = metrics_cnn["f1_train_average"].max()
max_f1_test = metrics_cnn["f1_test_average"].max()

max_f1_test, max_f1_train

(np.float64(0.6551872819161952), np.float64(0.973445018550672))